In [1]:
import findspark
import pandas as pd
findspark.init()
import pyspark
from pyspark.sql import SparkSession



spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 23:30:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [31]:
spark.version

'3.3.2'

In [2]:
fhv_df = pd.read_csv('data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz')


In [5]:
fhv_df.head()


,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014


In [7]:
from pyspark.sql import types

fhv_schema = types.StructType([
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropOff_datetime", types.TimestampType(), True),
    types.StructField("PUlocationID", types.IntegerType(), True),
    types.StructField("DOlocationID", types.IntegerType(), True),
    types.StructField("SR_Flag", types.StringType(), True),
    types.StructField("Affiliated_base_number", types.StringType(), True)
])

In [8]:
print('processing data for 2019/10')

input_path = 'data/raw/fhv/2019/10/'
output_path = 'data/pq/fhv/2019/10/'

df_fhv= spark.read \
    .option("header", "true") \
    .schema(fhv_schema) \
    .csv(input_path)

df_fhv \
    .repartition(6) \
    .write.parquet(output_path)

processing data for 2019/10


In [2]:
df_zones = spark.read.parquet('zones/')
df_fhv = spark.read.parquet('data/pq/fhv/2019/10/')

In [13]:
df_joined = df_fhv.join(df_zones,df_fhv.PUlocationID == df_zones.LocationID)

In [15]:
df_fhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   null|                  null|
|              B02429|2019-10-21 04:15:47|2019-10-21 04:36:04|         264|         264|   null|                B02429|
|              B01482|2019-10-19 12:00:00|2019-10-19 12:20:00|         264|         264|   null|                B01482|
|              B03015|2019-10-11 14:28:00|2019-10-11 14:32:44|         264|         216|   null|                B03015|
|              B01529|2019-10-21 18:00:26|2019-10-21 18:07:21|         264|          80|   null|                B01529|
|              B00477|2019-10-03 19:30:3

In [3]:
df_fhv.createOrReplaceTempView('fvh_trips')
df_zones.createOrReplaceTempView('zones')

In [5]:
df_result = spark.sql( """
 SELECT * FROM fvh_trips
 LIMIT 100;
 """)

In [6]:
df_result.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02784|2019-10-01 09:55:38|2019-10-01 10:05:43|          89|          85|   null|                  null|
|              B02429|2019-10-21 04:15:47|2019-10-21 04:36:04|         264|         264|   null|                B02429|
|              B01482|2019-10-19 12:00:00|2019-10-19 12:20:00|         264|         264|   null|                B01482|
|              B03015|2019-10-11 14:28:00|2019-10-11 14:32:44|         264|         216|   null|                B03015|
|              B01529|2019-10-21 18:00:26|2019-10-21 18:07:21|         264|          80|   null|                B01529|
|              B00477|2019-10-03 19:30:3

In [15]:
spark.sql( """
 SELECT count(*) FROM fvh_trips
 where  date_trunc('day',pickup_datetime) ='2019-10-15'
 LIMIT 100;
 """).show()

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [30]:
spark.sql( """
 SELECT ((dropOff_datetime  - pickup_datetime)* 24)  FROM fvh_trips
 order by 1 desc
 Limit 100;
 """).show()

+-------------------------------------------+
|((dropOff_datetime - pickup_datetime) * 24)|
+-------------------------------------------+
|                       INTERVAL '631152 ...|
|                       INTERVAL '631152 ...|
|                       INTERVAL '87672 1...|
|                       INTERVAL '70128 0...|
|                       INTERVAL '8794 00...|
|                       INTERVAL '8784 04...|
|                       INTERVAL '1464 12...|
|                       INTERVAL '1056 19...|
|                       INTERVAL '1056 06...|
|                       INTERVAL '793 13:...|
|                       INTERVAL '793 09:...|
|                       INTERVAL '793 07:...|
|                       INTERVAL '792 23:...|
|                       INTERVAL '792 23:...|
|                       INTERVAL '792 20:...|
|                       INTERVAL '792 19:...|
|                       INTERVAL '792 18:...|
|                       INTERVAL '792 18:...|
|                       INTERVAL '

In [25]:
spark.sql("""
SELECT * FROM zones
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [27]:
spark.sql( """
 SELECT z.Zone, count(*) FROM fvh_trips f
 inner join zones z on f.PUlocationID  = z.LocationID
 group by z.Zone
 order by 2 asc
 ;
 """).show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
|        Battery Park|      15|
|Saint Michaels Ce...|      23|
|Breezy Point/Fort...|      25|
|Marine Park/Floyd...|      26|
|        Astoria Park|      29|
|    Inwood Hill Park|      39|
|       Willets Point|      47|
|Forest Park/Highl...|      53|
|  Brooklyn Navy Yard|      57|
|        Crotona Park|      62|
|        Country Club|      77|
|     Freshkills Park|      89|
|       Prospect Park|      98|
|     Columbia Street|     105|
|  South Williamsburg|     110|
+--------------------+--------+
only showing top 20 rows

